In [3]:
import cv2

idx = 0
while True:
    cap = cv2.VideoCapture(idx)
    if not cap.isOpened():
        break
    cap.release()
    print(f"camera {idx} is available") 
    idx += 1




camera 0 is available
camera 1 is available
camera 2 is available
